In [2]:
import tensorflow as tf
import numpy as np

2025-11-27 15:47:46.901358: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-27 15:47:46.972515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 15:47:48.712270: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-27 15:47:48.712270: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [15]:
from pathlib import Path
import sys

# notebook is at music_classifier/notebooks; add the sibling scripts/ dir
sys.path.insert(0, str((Path.cwd() / '..' / 'scripts').resolve()))

# Now import as if preprocess.py were in the notebook folder
from preprocess import load_dataset
from transforms import apply_random_transforms

In [3]:
model_path = '/media/suraj/SSD/programming/Thesis/aigm-classifier/models/audio_classifier_model.keras'
model = tf.keras.models.load_model(model_path)
print('Loaded model. input shape:', model.input_shape)

2025-11-27 15:47:53.542242: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Loaded model. input shape: (None, 128, 128, 1)


In [8]:
model.layers    

[<Conv2D name=conv2d, built=True>,
 <MaxPooling2D name=max_pooling2d, built=True>,
 <Conv2D name=conv2d_1, built=True>,
 <MaxPooling2D name=max_pooling2d_1, built=True>,
 <Conv2D name=conv2d_2, built=True>,
 <MaxPooling2D name=max_pooling2d_2, built=True>,
 <Flatten name=flatten, built=True>,
 <Dense name=dense, built=True>,
 <Dropout name=dropout, built=True>,
 <Dense name=dense_1, built=True>]

In [16]:
# Discover audio files under data/testset/real/
audio_patterns = ['**/*.wav','**/*.mp3','**/*.flac','**/*.ogg']
data_root = Path('data/testset/real')
files = []
for pat in audio_patterns:
    files += list(data_root.glob(pat))
files = [p for p in files if p.is_file()]
print(f'Found {len(files)} audio files')

def prepare_input(y, sr, model):
    # Determine target mel bins and time frames from model input if possible
    input_shape = model.input_shape
    if len(input_shape) == 4:
        _, freq, time, ch = input_shape
    elif len(input_shape) == 3:
        _, freq, time = input_shape
        ch = None
    else:
        raise RuntimeError(f'Unexpected model.input_shape: {input_shape}')
    freq = freq or 128
    # Recompute mel spectrogram with matching n_mels
    mel = compute_mel(y, n_mels=freq)
    # Pad/truncate time axis to 'time' if provided
    if time is not None and mel.shape[1] != time:
        if mel.shape[1] < time:
            pad_width = time - mel.shape[1]
            mel = np.pad(mel, ((0,0),(0,pad_width)), mode='constant', constant_values=mel.min())
        else:
            mel = mel[:, :time]
    X = mel.astype('float32')
    if ch is not None:
        X = X[:, :, np.newaxis]
    return X

def evaluate_file(path, model):
    y, sr = load_audio(str(path), sr=22050)
    # If stereo, prefer mono for model input unless model expects 2 channels
    try:
        y_t = apply_random_transforms(y, sr)
    except Exception as e:
        print('Transform failed, using original audio:', e)
        y_t = y
    X = prepare_input(y_t, sr, model)
    pred = model.predict(np.expand_dims(X, 0))
    return pred, X.shape

# Sample and run transforms+inference
n = min(10, len(files)) if len(files) > 0 else 0
sample_files = random.sample(files, n) if n>0 else []
results = []
for p in sample_files:
    try:
        pred, shape = evaluate_file(p, model)
        results.append((str(p), pred.tolist(), shape))
        print(p, '->', pred, 'input shape', shape)
    except Exception as e:
        print('Error processing', p, e)

# Save results
import json
out = Path('results/metrics')
out.mkdir(parents=True, exist_ok=True)
with open(out / 'transform_evaluation.json', 'w') as f:
    json.dump(results, f, indent=2)
print('Saved', len(results), 'results to', out / 'transform_evaluation.json')

Found 0 audio files
Saved 0 results to results/metrics/transform_evaluation.json
